# Power Density Data

A power density database has been provided by Wilmarth, Stimac and Ganefiantio for inclusion in this repository and the application. 

Maxwell Wilmarth, James Stimac and Gugi Ganefianto (2020) Power Density in Geothermal Fields, 2020 Update, Proceedings World Geothermal Congress 2020+1, Reykjavik, Iceland, April - October 2021.

Wilmarth, M. and Stimac, J.: Power Density In Geothermal Fields, Proceedings, World Geothermal Conference (2015).

Database paramaters

Power density is the power (MWe) per area (km2) that is generated. 

**Power capacity**

The Wilmarth et al (2020, 2015) database uses net power output in MWe that is based on published data or information provided by the operating company directly to the authors. This contrasts with installed capacity, which is typically publicly reported and is included in the [Think Geoenergy power plant map](https://www.thinkgeoenergy.com/map/). The gap between installed capacity and net generation may be related to plant efficiency or parasitic load. Alternatively, resource factors may influence, such as pressure decline or enthalpy evolution, that means that the power plant does not generate at full capacity.  

**Area**

Wilmarth et al (2020, 2015) use 500 m buffer around the production wells to define the reserver area. This is important to keep in mind when using the power density plot for resource capacity estimates: The power density values can not be blindly applied to the full extent of the resistivity anomaly or even an area defined by the maximum extent of geothermal manifestations. Instead, the area of productive resource in the subsurface must be defined using a conceptual model, ideally with each of the P10, P50 and P90 areas considered.    

**Average temperature**

Geothermal resources contain a range of temperatures in the subsurface, their production temperature/enthalpy may evolve though time, and reliable reporting of resource temperature may be difficult to obtain. It is therefore unsurprising that Wilmarth et al. (2020) cites average reservoir temperature as the most difficult parameter to estimate. 

## Notebook purpose

This notebook has been developed to explore the power density database and determine how best to plot the data in the Streamlit application. 
